In [ ]:
import numpy as np
import torch 
import Phi

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
class Score_Matching_Landau():
    def __init__(self, v):
        
        self.v = v.to(device) # velocity
        self.nex = v.shape[0] # number of particles
        self.d = v.shape[1] # dimension

        # neural network
        self.net = Phi.Phi(nTh=3, m=20, d=self.d).to(device)

        # optimizer Adam
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=1e-3)
        self.iter = 0

    def proj(self, v_diff, norm):
        v_temp = v_diff[:,None,:]
        proj = (norm**2)[:,:,None] * torch.eye(self.d).to(device) - v_temp.permute(0,2,1) * v_temp
        return proj

    def loss(self): # score-matching
        self.optimizer.zero_grad()

        u, div_u = self.net.trHess(self.v, justGrad=False)
        loss = ( torch.norm(u)**2 + 2*torch.sum(div_u) ) / self.nex
        loss.backward()

        error = self.error(self.v)
        if self.iter % 100 == 0:
            print('Iter %d, Loss: %.5e' % (self.iter, loss.item()))
            print(error)

        self.iter += 1
        return loss
    

    def train(self):
        while self.iter <= 15000:
            self.optimizer.step(self.loss)
        
    def predict(self, v):
        u = self.net.trHess(v, justGrad=True)
        return u

    def error(self, v):
        error = torch.sum((self.predict(v) + v) / v) / self.nex
        return error

In [ ]:
nex = 10000   
d = 1
v = torch.randn(nex,d)

model = Score_Matching_Landau(v)
model.train()